### Second Project

In [3]:
using Plots, ROCAnalysis, GLM, MLBase, Lathe, Distributions, LaTeXStrings, CSV, DataFrames, StatsBase, Missings, XLSX
using IJulia: readprompt

#### Q1

In [8]:
df = DataFrame(XLSX.readtable("crypto_currency.xlsx", "Sheet1")...)

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,39K
2,Ethereum,ETH,360B,3K
3,Ripple,XRP,50B,0.7
4,Solana,SOL,34B,100
5,Terra,LUNA,32B,91
6,Cardano,ADA,30B,0.9
7,Avalanche,AVAX,20B,73
8,Polkadot,DOT,19B,19
9,Dogecoin,DOGE,18B,0.13


In [19]:
names(df)

4-element Vector{String}:
 "name"
 "ticket"
 "market cap"
 "last price(\$)"

In [110]:
selected_rows = findall(x->x ∈ sort(df."market cap", rev=true)[1:5], df."market cap")
top_five=df[selected_rows,:]

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,39K
2,Ethereum,ETH,360B,3K
3,Ripple,XRP,50B,0.7
4,Solana,SOL,34B,100
5,Terra,LUNA,32B,91


In [108]:
for i in top_five."last price(\$)"
    if typeof(i) == String && contains(i, "K") == true
        replace!(top_five."last price(\$)", i=>parse(Int, chop(i))*1000)
    end
end
top_five

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,-20.9895
2,Ethereum,ETH,360B,0.0474742
3,Ripple,XRP,50B,0.0471464
4,Solana,SOL,34B,0.0473507
5,Terra,LUNA,32B,0.0475091


In [101]:
sum(top_five."last price(\$)")

42191.7

##### *sclaing data*
$$
x := \frac{x - \mu}{\sigma}
$$

In [107]:

replace!(x -> (x - mean(top_five."last price(\$)"))/var(top_five."last price(\$)"), top_five."last price(\$)")
top_five

,name,ticket,market cap,last price($)
,Any,Any,Any,Any
1,Bitcoin,BTC,750B,-20.9895
2,Ethereum,ETH,360B,0.0474742
3,Ripple,XRP,50B,0.0471464
4,Solana,SOL,34B,0.0473507
5,Terra,LUNA,32B,0.0475091


## Mainnnn


In [53]:
df = DataFrame("Date" => reverse!(vec(convert(Array, select!(CSV.read("BTC-USD.csv", DataFrame), "Date")))),
     "Bitcoin" => reverse!(vec(convert(Array, select!(CSV.read("BTC-USD.csv", DataFrame), "Close")))),
     "Ethereum"=> reverse!(vec(convert(Array, select!(CSV.read("ETH-USD.csv", DataFrame), "Close")))), 
     "Ripple"=> reverse!(vec(convert(Array, select!(CSV.read("XRP-USD.csv", DataFrame), "Close")))),
     "Solana"=> reverse!(vec(convert(Array, select!(CSV.read("SOL-USD.csv", DataFrame), "Close")))), 
     "Terra"=> reverse!(push!(vec(convert(Array, select!(CSV.read("LUNA-USD.csv", DataFrame), "Close"))), "null")),
     "Cardano"=> reverse!(vec(convert(Array, select!(CSV.read("ADA-USD.csv", DataFrame), "Close")))), 
     "Avalanche"=> reverse!(vec(convert(Array, select!(CSV.read("AVAX-USD.csv", DataFrame), "Close")))), 
     "Polkadot"=> reverse!(vec(convert(Array, select!(CSV.read("DOT-USD.csv", DataFrame), "Close")))),
     "Dogecoin"=> reverse!(vec(convert(Array, select!(CSV.read("DOGE-USD.csv", DataFrame), "Close")))), 
     "Polygon"=> reverse!(vec(convert(Array, select!(CSV.read("MATIC-USD.csv", DataFrame), "Close")))))

,Date,Bitcoin,Ethereum,Ripple,Solana,Terra,Cardano,Avalanche
,Date…,Float64,Float64,Float64,Float64,String15,Float64,Float64
1,2022-05-16,29811.8,2035.94,0.420961,55.0012,null,0.568933,33.295
2,2022-05-15,31305.1,2145.71,0.447371,58.8113,null,0.5975,36.5423
3,2022-05-14,30101.3,2056.27,0.427774,52.4172,null,0.539358,34.0825
4,2022-05-13,29283.1,2014.42,0.423417,48.5936,null,0.528877,32.1364
5,2022-05-12,29047.8,1961.7,0.385206,44.6836,null,0.473746,30.3575
6,2022-05-11,28936.4,2072.11,0.413376,50.2109,1.192089,0.5128,30.9629
7,2022-05-10,31022.9,2343.51,0.513626,66.7662,0.005584,0.628963,44.5603
8,2022-05-09,30297.0,2245.43,0.49294,63.2697,0.003636,0.610088,41.972
9,2022-05-08,34059.3,2517.46,0.566527,75.2202,0.003747,0.739563,51.5212


In [66]:
for i in names(df)
    replace!(x -> (x - mean(df.i))/var(df.i), df.i)
end


LoadError: ArgumentError: column name :i not found in the data frame

### Q2